In [1]:
import torch, detectron2
!nvcc --version
TORCH_VERSION = ".".join(torch.__version__.split(".")[:2])
CUDA_VERSION = torch.__version__.split("+")[-1]
print("torch: ", TORCH_VERSION, "; cuda: ", CUDA_VERSION)
print("detectron2:", detectron2.__version__)

/home/djjin/anaconda3/envs/conda_visual_HPE/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2022 NVIDIA Corporation
Built on Tue_May__3_18:49:52_PDT_2022
Cuda compilation tools, release 11.7, V11.7.64
Build cuda_11.7.r11.7/compiler.31294372_0
torch:  1.13 ; cuda:  cu117
detectron2: 0.6


In [2]:
import os
import sys
import logging
import argparse

pth = '/'.join(sys.path[0].split('/')[:-1])
sys.path.insert(0, pth)

from pprint import pprint
from PIL import Image
from copy import deepcopy
import numpy as np
np.random.seed(1)

home_dir = os.path.abspath(os.getcwd()+"/../")
sys.path.append(home_dir)
print(home_dir)

import warnings
warnings.filterwarnings(action='ignore')
# logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

import torch
from torchvision import transforms
from torch.nn import functional as F

from detectron2.data import MetadataCatalog
from detectron2.utils.colormap import random_color
from detectron2.structures import Boxes, ImageList, Instances, BitMasks, BoxMode

from hdecoder.BaseModel import BaseModel
from hdecoder import build_model

from utils.arguments import load_opt_command
from utils.misc import hook_metadata, hook_switcher, hook_opt
from utils.distributed import init_distributed
from utils.arguments import load_opt_from_config_files, load_config_dict_to_opt

/home/djjin/Mygit/X-Decoder


Invalid MIT-MAGIC-COOKIE-1 key

In [3]:
parser = argparse.ArgumentParser(description='Pretrain or fine-tune models for NLP tasks.')
parser.add_argument('--command', default="evaluate", help='Command: train/evaluate/train-and-evaluate')
parser.add_argument('--conf_files', nargs='+', help='Path(s) to the config file(s).')
parser.add_argument('--user_dir', help='Path to the user defined module for tasks (models, criteria), optimizers, and lr schedulers.')
parser.add_argument('--config_overrides', nargs='*', help='Override parameters on config with a json style string, e.g. {"<PARAM_NAME_1>": <PARAM_VALUE_1>, "<PARAM_GROUP_2>.<PARAM_SUBGROUP_2>.<PARAM_2>": <PARAM_VALUE_2>}. A key with "." updates the object in the corresponding nested dict. Remember to escape " in command line.')
parser.add_argument('--overrides', help='arguments that used to override the config file in cmdline', nargs=argparse.REMAINDER)

cmdline_args = parser.parse_args('')
# cmdline_args.conf_files = [os.path.join(home_dir, "configs/xdecoder/svlp_focalt_lang.yaml")]
cmdline_args.conf_files = [os.path.join(home_dir, "configs/xdecoder/vcoco.yaml")]
cmdline_args.overrides = ['WEIGHT', 'true', 'RESUME_FROM', '../checkpoints/xdecoder_focalt_best_openseg.pt'] 

opt = load_opt_from_config_files(cmdline_args.conf_files)

keys = [cmdline_args.overrides[idx*2] for idx in range(len(cmdline_args.overrides)//2)]
vals = [cmdline_args.overrides[idx*2+1] for idx in range(len(cmdline_args.overrides)//2)]
print(keys, vals)
vals = [val.replace('false', '').replace('False','') if len(val.replace(' ', '')) == 5 else val for val in vals]
types = []
for key in keys:
    key = key.split('.')
    ele = opt.copy()
    while len(key) > 0:
        ele = ele[key.pop(0)]
    types.append(type(ele))

config_dict = {x:z(y) for x,y,z in zip(keys, vals, types)}

load_config_dict_to_opt(opt, config_dict)
for key, val in cmdline_args.__dict__.items():
    if val is not None:
        opt[key] = val
opt = init_distributed(opt)

['WEIGHT', 'RESUME_FROM'] ['true', '../checkpoints/xdecoder_focalt_best_openseg.pt']


In [4]:
pprint(opt["MODEL"]["FREEZE_NAME"])

['backbone', 'encoder']


In [5]:
fix_param = opt['SOLVER'].get('FIX_PARAM',{})
fix_param

{'backbone': True, 'encoder': True}

In [6]:
# pretrained_pth = os.path.join(opt['RESUME_FROM'])
# print(pretrained_pth)
# model = BaseModel(opt, build_model(opt)).from_pretrained(pretrained_pth).train().cuda()

In [7]:
# model

In [8]:
# t = []
# t.append(transforms.Resize(800, interpolation=Image.BICUBIC))
# transform = transforms.Compose(t)
# model.eval()
# model.model.backbone

# pixel_mean = torch.Tensor([123.675, 116.280, 103.530]).view(-1, 1, 1).cuda()
# pixel_std = torch.Tensor([58.395, 57.120, 57.375]).view(-1, 1, 1).cuda()

# num_layers = opt['MODEL']['BACKBONE']['FOCAL']['DEPTHS']
# out_indices = opt['MODEL']['BACKBONE']['FOCAL']['OUT_INDICES']
# focal_levels = opt['MODEL']['BACKBONE']['FOCAL']['FOCAL_LEVELS']
# num_layers, out_indices, focal_levels
# image_pth = '../images/animals.png'

# with torch.no_grad():
#     image_ori = Image.open(image_pth).convert('RGB')
#     width = image_ori.size[0]
#     height = image_ori.size[1]
#     image = transform(image_ori)
#     image = np.asarray(image)
#     image_ori = np.asarray(image_ori)
#     images = torch.from_numpy(image.copy()).permute(2,0,1).cuda()
#     batch_inputs = [{'image': images, 'height': height, 'width': width}]
#     # outputs = model.forward(batch_inputs)

#     images = [x["image"].to("cuda") for x in batch_inputs]
#     images = [(x - pixel_mean) / pixel_std for x in images]
    
#     images = ImageList.from_tensors(images, 32)
#     print(f"Image shape: {images.tensor.shape}")

#     ######################################################
#     ## Backbone
#     outputs = model.model.backbone(images.tensor)
#     ## Encoder
#     transformer_encoder_features, _ = model.model.hoid_head.encoder.forward_features(outputs)
#     print(transformer_encoder_features.shape)

```sh
CUDA_VISIBLE_DEVICES=0 python entry.py train \
            --conf_files configs/xdecoder/segvlp_focalt_lang.yaml \
            --overrides \
            COCO.INPUT.IMAGE_SIZE 1024 \
            MODEL.DECODER.CAPTIONING.ENABLED True \
            MODEL.DECODER.RETRIEVAL.ENABLED True \
            MODEL.DECODER.GROUNDING.ENABLED True \
            MODEL.DECODER.CAPTIONING_WEIGHT 8 \
            MODEL.DECODER.RETRIEVAL_WEIGHT 8 \
            MODEL.DECODER.TOP_CAPTIONING_LAYERS 3 \
            MODEL.DECODER.TOP_RETRIEVAL_LAYERS 3 \
            MODEL.DECODER.TOP_GROUNDING_LAYERS 6 \
            COCO.TEST.BATCH_SIZE_TOTAL 1 \
            COCO.TRAIN.BATCH_SIZE_TOTAL 1 \
            COCO.TRAIN.BATCH_SIZE_PER_GPU 1 \
            VLP.TEST.BATCH_SIZE_TOTAL 2 \
            VLP.TRAIN.BATCH_SIZE_TOTAL 2 \
            VLP.TRAIN.BATCH_SIZE_PER_GPU 2 \
            MODEL.DECODER.HIDDEN_DIM 512 \
            MODEL.ENCODER.CONVS_DIM 512 \
            MODEL.ENCODER.MASK_DIM 512 \
            FP16 True \
            WEIGHT True \
            RESUME_FROM checkpoints/xdecoder_focalt_last.pt
```

In [9]:
from trainer import HDecoder_Trainer as Trainer
trainer = Trainer(opt)
trainer.train()

----------------
MPI Adapter data
----------------
environment info: no MPI
init method url: tcp://127.0.0.1:36873
world size: 1
local size: 1
rank: 0
local rank: 0
master address: 127.0.0.1
master port: 36873
----------------


Dataset length: 5400
num of train samples: 2700


*UNLOADED* hoid_head.hoi_decoder.hopd_decoder.layers.0.linear1.bias, Model Shape: torch.Size([2048])
*UNLOADED* hoid_head.hoi_decoder.hopd_decoder.layers.0.linear1.weight, Model Shape: torch.Size([2048, 512])
*UNLOADED* hoid_head.hoi_decoder.hopd_decoder.layers.0.linear2.bias, Model Shape: torch.Size([512])
*UNLOADED* hoid_head.hoi_decoder.hopd_decoder.layers.0.linear2.weight, Model Shape: torch.Size([512, 2048])
*UNLOADED* hoid_head.hoi_decoder.hopd_decoder.layers.0.multihead_attn.in_proj_bias, Model Shape: torch.Size([1536])
*UNLOADED* hoid_head.hoi_decoder.hopd_decoder.layers.0.multihead_attn.in_proj_weight, Model Shape: torch.Size([1536, 512])
*UNLOADED* hoid_head.hoi_decoder.hopd_decoder.layers.0.multihead_attn.out_proj.bias, Model Shape: torch.Size([512])
*UNLOADED* hoid_head.hoi_decoder.hopd_decoder.layers.0.multihead_attn.out_proj.weight, Model Shape: torch.Size([512, 512])
*UNLOADED* hoid_head.hoi_decoder.hopd_decoder.layers.0.norm1.bias, Model Shape: torch.Size([512])
*UNLOAD

epochs[     0] optim steps[1/2700] learning rate[default: 1.00000e-04] train loss[loss_obj_ce: 4.49451/4.49451, obj_class_error: 100.00000/100.00000, loss_verb_ce: 48.98612/48.98612, loss_sub_bbox: 0.76917/0.76917, loss_obj_bbox: 1.17902/1.17902, loss_sub_giou: 0.61277/0.61277, loss_obj_giou: 1.24644/1.24644, obj_cardinality_error: 97.00000/97.00000, loss_obj_ce_0: 4.66422/4.66422, loss_verb_ce_0: 51.61600/51.61600, loss_sub_bbox_0: 0.74749/0.74749, loss_obj_bbox_0: 1.16673/1.16673, loss_sub_giou_0: 0.60700/0.60700, loss_obj_giou_0: 1.24255/1.24255, obj_cardinality_error_0: 97.00000/97.00000, loss_obj_ce_1: 4.54103/4.54103, loss_verb_ce_1: 49.83380/49.83380, loss_sub_bbox_1: 0.75822/0.75822, loss_obj_bbox_1: 1.17844/1.17844, loss_sub_giou_1: 0.61127/0.61127, loss_obj_giou_1: 1.24752/1.24752, obj_cardinality_error_1: 97.00000/97.00000] items per batch[1] items per second[0.68] total items[1] mini batches[     1] memory[976] epoch remaining[1:06:12]
epochs[     0] optim steps[2/2700] lea

Saving checkpoint...


...

epochs[    22] optim steps[100/2700] learning rate[default: 1.00000e-04] train loss[loss_obj_ce: 0.37189/0.76943, obj_class_error: 33.33334/55.85815, loss_verb_ce: 1.03911/1.02653, loss_sub_bbox: 0.26017/0.39442, loss_obj_bbox: 0.30315/0.37063, loss_sub_giou: 0.28858/0.57722, loss_obj_giou: 0.78111/0.79870, obj_cardinality_error: 1.50000/2.70100, loss_obj_ce_0: 0.37767/0.77545, loss_verb_ce_0: 1.10578/1.08241, loss_sub_bbox_0: 0.31281/0.39634, loss_obj_bbox_0: 0.61276/0.37415, loss_sub_giou_0: 0.30119/0.57913, loss_obj_giou_0: 0.90805/0.79965, obj_cardinality_error_0: 1.50000/2.69793, loss_obj_ce_1: 0.31180/0.76855, loss_verb_ce_1: 1.09809/1.05935, loss_sub_bbox_1: 0.23376/0.39333, loss_obj_bbox_1: 0.54999/0.36874, loss_sub_giou_1: 0.26150/0.57504, loss_obj_giou_1: 0.78636/0.79682, obj_cardinality_error_1: 1.50000/2.70115] items per batch[1] items per second[0.02] total items[59500] mini batches[ 59500] memory[1700] epoch remaining[0:18:16]
epochs[    22] optim steps[200/2700] learning

Saving checkpoint...


------------------------------------------------------------
               hold_obj: #GTs = 0025, AP = 0.0000
                  stand: #GTs = 0040, AP = 0.0045
              sit_instr: #GTs = 0014, AP = 0.0000
             ride_instr: #GTs = 0006, AP = 0.0000
                   walk: #GTs = 0003, AP = 0.0000
               look_obj: #GTs = 0026, AP = 0.0001
              hit_instr: #GTs = 0002, AP = 0.0000
                hit_obj: #GTs = 0002, AP = 0.0000
                eat_obj: #GTs = 0005, AP = 0.0000
              eat_instr: #GTs = 0005, AP = 0.0000
             jump_instr: #GTs = 0004, AP = 0.0000
              lay_instr: #GTs = 0002, AP = 0.0000
    talk_on_phone_instr: #GTs = 0003, AP = 0.0000
              carry_obj: #GTs = 0005, AP = 0.0000
              throw_obj: #GTs = 0004, AP = 0.0000
              catch_obj: #GTs = 0004, AP = 0.0000
              cut_instr: #GTs = 0003, AP = 0.0000
                cut_obj: #GTs = 0003, AP = 0.0000
                    run: #GTs = 0008, A

Finished saving checkpoint and model to /home/djjin/Mygit/X-Decoder/data/output/test/00062100.


KeyboardInterrupt: 

In [ ]:
for name, param in trainer.models['default'].named_parameters():
    print(name, param.requires_grad)